In [ ]:
# TODO
# Set up spark session ✔
# Import single-line JSON files ✔
    # Optimize ==> Create a Function to Import all of these files and assign names ot them; perhaps using wrappers?

# Data validation
    # Check if any null values ✔
    # Customer according to specs:




## Required Imports
---

In [1]:
import pandas as pd
import pyspark 
import pyspark.pandas as ps
from pyspark.sql.functions import udf
from pyspark.sql import SparkSession
from pyspark.sql.types import \
    IntegerType, \
    StringType, \
    TimestampType, \
    DoubleType


c:\Users\GzuzC\OneDrive\Documents\Notes\Personal Studies\Workbooks and Worksheets\Python\PerScholas Data Engineering\perscholas_capstone_project\ignore\ETL-capstone-env\lib\site-packages\pyspark\pandas\__init__.py:49: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [2]:
session = SparkSession.builder.appName('Credit Card ETL').getOrCreate()


In [3]:
session

## Extract:
---
- Customer
- Branch
- Credit


In [4]:
customer_df = session.read.load('../data/credit_card_dataset/cdw_sapp_custmer.json', format='json')

In [4]:
branch_df = session.read.load('../data/credit_card_dataset/cdw_sapp_branch.json', format='json')

In [61]:
credit_df = session.read.load('../data/credit_card_dataset/cdw_sapp_credit.json', format='json')

In [232]:
#customer_df_dirty_backup = customer_df
branch_df_dirty_backup = branch_df
credit_df_dirty_backup = credit_df

## Transform
---

### Data Validation
- Ensure Correct Schema, View Current DataTypes
- Determine if any null values

In [235]:
print("Customer DataFrame:\n")
#customer_df.printSchema()
print("\nBranch DataFrame:\n")
branch_df.printSchema()
print("\nCredit DataFrame:\n")
branch_df.printSchema()




Customer DataFrame:


Branch DataFrame:

root
 |-- BRANCH_CITY: string (nullable = true)
 |-- BRANCH_CODE: long (nullable = true)
 |-- BRANCH_NAME: string (nullable = true)
 |-- BRANCH_PHONE: string (nullable = true)
 |-- BRANCH_STATE: string (nullable = true)
 |-- BRANCH_STREET: string (nullable = true)
 |-- BRANCH_ZIP: long (nullable = true)
 |-- LAST_UPDATED: string (nullable = true)


Credit DataFrame:

root
 |-- BRANCH_CITY: string (nullable = true)
 |-- BRANCH_CODE: long (nullable = true)
 |-- BRANCH_NAME: string (nullable = true)
 |-- BRANCH_PHONE: string (nullable = true)
 |-- BRANCH_STATE: string (nullable = true)
 |-- BRANCH_STREET: string (nullable = true)
 |-- BRANCH_ZIP: long (nullable = true)
 |-- LAST_UPDATED: string (nullable = true)



In [9]:
def null_checker(dataframe):
    original_count= dataframe.count()
    drop_na = dataframe.dropna()
    new_count = drop_na.count()
    if new_count != original_count:
        print(f"Original Number of Rows: {original_count} \
            \nNew Number of Rows: {new_count}\n")
    else:
        print("No nulls, no changes made\n")


In [10]:
null_checker(customer_df)
null_checker(branch_df)
null_checker(credit_df)

No nulls, no changes made

No nulls, no changes made

No nulls, no changes made



### Casting Correct Data Types

In [86]:
#test = customer_df.select(customer_df['SSN'], customer_df['SSN'].cast("INT"))

In [11]:
customer_df.createOrReplaceTempView("customer_df")

In [25]:
# this technique is the fastest way to convert datatypes without running a loop on .withColumn 
# to prevent potential performance loss or crashing
customer_df_transformed = customer_df.withColumns({'SSN':customer_df['SSN'].cast(IntegerType()), 
                                  'CUST_ZIP':customer_df['CUST_ZIP'].cast(IntegerType()),
                                  'CUST_PHONE':customer_df['CUST_PHONE'].cast(StringType()),
                                  'LAST_UPDATED':customer_df['LAST_UPDATED'].cast(TimestampType())})



In [13]:
print("Transformed:\n")
customer_df_transformed.printSchema()
print("\nOriginal:\n")
customer_df.printSchema()

Transformed:

root
 |-- APT_NO: string (nullable = true)
 |-- CREDIT_CARD_NO: string (nullable = true)
 |-- CUST_CITY: string (nullable = true)
 |-- CUST_COUNTRY: string (nullable = true)
 |-- CUST_EMAIL: string (nullable = true)
 |-- CUST_PHONE: string (nullable = true)
 |-- CUST_STATE: string (nullable = true)
 |-- CUST_ZIP: integer (nullable = true)
 |-- FIRST_NAME: string (nullable = true)
 |-- LAST_NAME: string (nullable = true)
 |-- LAST_UPDATED: timestamp (nullable = true)
 |-- MIDDLE_NAME: string (nullable = true)
 |-- SSN: integer (nullable = true)
 |-- STREET_NAME: string (nullable = true)


Original:

root
 |-- APT_NO: string (nullable = true)
 |-- CREDIT_CARD_NO: string (nullable = true)
 |-- CUST_CITY: string (nullable = true)
 |-- CUST_COUNTRY: string (nullable = true)
 |-- CUST_EMAIL: string (nullable = true)
 |-- CUST_PHONE: long (nullable = true)
 |-- CUST_STATE: string (nullable = true)
 |-- CUST_ZIP: string (nullable = true)
 |-- FIRST_NAME: string (nullable = true)


### Transforming Single Columns

In [24]:
import time
# I want to determine if using a pandas api layer on spark would be faster while running locally, or a direct pandas dataframe
# I confirmed that it is just faster to use a pandas dataframe while running locally as opposed to the pandas API layer

In [26]:
customer_df_transformed_backup = customer_df_transformed

In [27]:
customer_df_transformed_pandas_api = customer_df_transformed.pandas_api()

In [70]:
customer_df_transformed_pandas_direct = customer_df_transformed.toPandas()

c:\Users\GzuzC\OneDrive\Documents\Notes\Personal Studies\Workbooks and Worksheets\Python\PerScholas Data Engineering\perscholas_capstone_project\ignore\ETL-capstone-env\lib\site-packages\pyspark\sql\pandas\conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


In [51]:
start_api = time.time()
middle_name_api = customer_df_transformed_pandas_api['MIDDLE_NAME'].apply(lambda x: str(x).lower())
end_api = time.time()

c:\Users\GzuzC\OneDrive\Documents\Notes\Personal Studies\Workbooks and Worksheets\Python\PerScholas Data Engineering\perscholas_capstone_project\ignore\ETL-capstone-env\lib\site-packages\pyspark\pandas\internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  fields = [
c:\Users\GzuzC\OneDrive\Documents\Notes\Personal Studies\Workbooks and Worksheets\Python\PerScholas Data Engineering\perscholas_capstone_project\ignore\ETL-capstone-env\lib\site-packages\pyspark\sql\pandas\conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [52]:
print(f"{end_api - start_api}")

0.15231633186340332


In [47]:
middle_name_api

0              wm
1         brendan
2        ezequiel
3           trina
4             may
5         ambrose
6           larry
7             ora
8          tracie
9         mitchel
10          denny
11       isabelle
12          henry
13         rickey
14      heriberto
15       angelita
16       dorothea
17      jefferson
18         maximo
19        arnulfo
20         ramiro
21        nanette
22       isabella
23          edith
24        jacklyn
25          lorna
26         debora
27          tomas
28       carolina
29        shannon
30         austin
31         lorrie
32           phil
33          britt
34        chasity
35        mildred
36            ted
37        gonzalo
38         harvey
39         andrew
40         corina
41           oren
42          cathy
43        raleigh
44          scott
45         santos
46       jennifer
47        rosario
48         leonor
49       laurence
50           erin
51          brain
52         antone
53       mercedes
54           aida
55        

In [53]:
start_pandas = time.time()
middle_name_pandas = customer_df_transformed_pandas_direct['MIDDLE_NAME'].apply(lambda x: str(x).lower())
end_pandas = time.time()

In [54]:
print(f"{end_pandas - start_pandas}")

0.0010008811950683594


In [57]:
print(f"{((end_api - start_api)/(end_pandas - start_pandas))} Times Faster")

152.18222963315864 Times Faster


In [71]:
customer_df_transformed_pandas_direct['MIDDLE_NAME'] = middle_name_pandas

In [72]:
customer_df_transformed_pandas_direct.head(2)

,APT_NO,CREDIT_CARD_NO,CUST_CITY,CUST_COUNTRY,CUST_EMAIL,CUST_PHONE,CUST_STATE,CUST_ZIP,FIRST_NAME,LAST_NAME,LAST_UPDATED,MIDDLE_NAME,SSN,STREET_NAME
0,656,4210653310061055,Natchez,United States,AHooper@example.com,1237818,MS,39120,Alec,Hooper,2018-04-21 12:49:02,wm,123456100,Main Street North
1,829,4210653310102868,Wethersfield,United States,EHolman@example.com,1238933,CT,6109,Etta,Holman,2018-04-21 12:49:02,brendan,123453023,Redwood Drive


In [95]:
pandas_backup = customer_df_transformed_pandas_direct  # clean this up later

In [99]:
customer_df_transformed_pandas_direct = pandas_backup

In [100]:
for name in ['FIRST_NAME', 'LAST_NAME']:
    customer_df_transformed_pandas_direct[name] = customer_df_transformed_pandas_direct[name].apply(lambda x: str(x).title())


In [111]:
customer_df_transformed_pandas_direct

,APT_NO,CREDIT_CARD_NO,CUST_CITY,CUST_COUNTRY,CUST_EMAIL,CUST_PHONE,CUST_STATE,CUST_ZIP,FIRST_NAME,LAST_NAME,LAST_UPDATED,MIDDLE_NAME,SSN,STREET_NAME
0,656,4210653310061055,Natchez,United States,AHooper@example.com,1237818,MS,39120,Alec,Hooper,2018-04-21 12:49:02,wm,123456100,Main Street North
1,829,4210653310102868,Wethersfield,United States,EHolman@example.com,1238933,CT,6109,Etta,Holman,2018-04-21 12:49:02,brendan,123453023,Redwood Drive
2,683,4210653310116272,Huntley,United States,WDunham@example.com,1243018,IL,60142,Wilber,Dunham,2018-04-21 12:49:02,ezequiel,123454487,12th Street East
3,253,4210653310195948,NewBerlin,United States,EHardy@example.com,1243215,WI,53151,Eugenio,Hardy,2018-04-21 12:49:02,trina,123459758,Country Club Road
4,301,4210653310356919,ElPaso,United States,WAyers@example.com,1242074,TX,79930,Wilfred,Ayers,2018-04-21 12:49:02,may,123454431,Madison Street
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
947,882,4210653399559239,SiouxCity,United States,FCastle@example.com,1238344,IA,51106,Freida,Castle,2018-04-21 12:49:02,jonah,123451589,Main Street South
948,470,4210653399650358,Summerville,United States,FBlock@example.com,1240420,SC,29483,Fern,Block,2018-04-21 12:49:02,aron,123457088,Glenwood Avenue
949,405,4210653399732638,Duluth,United States,DPruitt@example.com,1236149,GA,30096,Dylan,Pruitt,2018-04-21 12:49:02,lucas,123453259,Country Lane
950,15,4210653399859149,Rowlett,United States,EBeatty@example.com,1236886,TX,75088,Emery,Beatty,2018-04-21 12:49:02,susanna,123454047,8th Street


In [114]:
customer_df_transformed_pandas_direct['CUST_PHONE']
print(len(customer_df_transformed_pandas_direct['CUST_PHONE'].iloc[0])) # missing area code, 7nums

print(len(customer_df_transformed_pandas_direct['CUST_CITY'].unique()))  # 151 diff area codes, just 555 for now


7
151


In [115]:
customer_df_transformed_pandas_direct['CUST_PHONE'] = customer_df_transformed_pandas_direct['CUST_PHONE'].apply(lambda x: "(555)" + str(x))

In [116]:
customer_df_transformed_pandas_direct

,APT_NO,CREDIT_CARD_NO,CUST_CITY,CUST_COUNTRY,CUST_EMAIL,CUST_PHONE,CUST_STATE,CUST_ZIP,FIRST_NAME,LAST_NAME,LAST_UPDATED,MIDDLE_NAME,SSN,STREET_NAME
0,656,4210653310061055,Natchez,United States,AHooper@example.com,(555)1237818,MS,39120,Alec,Hooper,2018-04-21 12:49:02,wm,123456100,Main Street North
1,829,4210653310102868,Wethersfield,United States,EHolman@example.com,(555)1238933,CT,6109,Etta,Holman,2018-04-21 12:49:02,brendan,123453023,Redwood Drive
2,683,4210653310116272,Huntley,United States,WDunham@example.com,(555)1243018,IL,60142,Wilber,Dunham,2018-04-21 12:49:02,ezequiel,123454487,12th Street East
3,253,4210653310195948,NewBerlin,United States,EHardy@example.com,(555)1243215,WI,53151,Eugenio,Hardy,2018-04-21 12:49:02,trina,123459758,Country Club Road
4,301,4210653310356919,ElPaso,United States,WAyers@example.com,(555)1242074,TX,79930,Wilfred,Ayers,2018-04-21 12:49:02,may,123454431,Madison Street
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
947,882,4210653399559239,SiouxCity,United States,FCastle@example.com,(555)1238344,IA,51106,Freida,Castle,2018-04-21 12:49:02,jonah,123451589,Main Street South
948,470,4210653399650358,Summerville,United States,FBlock@example.com,(555)1240420,SC,29483,Fern,Block,2018-04-21 12:49:02,aron,123457088,Glenwood Avenue
949,405,4210653399732638,Duluth,United States,DPruitt@example.com,(555)1236149,GA,30096,Dylan,Pruitt,2018-04-21 12:49:02,lucas,123453259,Country Lane
950,15,4210653399859149,Rowlett,United States,EBeatty@example.com,(555)1236886,TX,75088,Emery,Beatty,2018-04-21 12:49:02,susanna,123454047,8th Street


In [119]:
customer_transformed_df = session.createDataFrame(customer_df_transformed_pandas_direct)

c:\Users\GzuzC\OneDrive\Documents\Notes\Personal Studies\Workbooks and Worksheets\Python\PerScholas Data Engineering\perscholas_capstone_project\ignore\ETL-capstone-env\lib\site-packages\pyspark\sql\pandas\conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
c:\Users\GzuzC\OneDrive\Documents\Notes\Personal Studies\Workbooks and Worksheets\Python\PerScholas Data Engineering\perscholas_capstone_project\ignore\ETL-capstone-env\lib\site-packages\pyspark\sql\pandas\conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [120]:
customer_transformed_df.printSchema()

root
 |-- APT_NO: string (nullable = true)
 |-- CREDIT_CARD_NO: string (nullable = true)
 |-- CUST_CITY: string (nullable = true)
 |-- CUST_COUNTRY: string (nullable = true)
 |-- CUST_EMAIL: string (nullable = true)
 |-- CUST_PHONE: string (nullable = true)
 |-- CUST_STATE: string (nullable = true)
 |-- CUST_ZIP: long (nullable = true)
 |-- FIRST_NAME: string (nullable = true)
 |-- LAST_NAME: string (nullable = true)
 |-- LAST_UPDATED: timestamp (nullable = true)
 |-- MIDDLE_NAME: string (nullable = true)
 |-- SSN: long (nullable = true)
 |-- STREET_NAME: string (nullable = true)



In [121]:
customer_transformed_df.show(2)

+------+----------------+------------+-------------+-------------------+------------+----------+--------+----------+---------+-------------------+-----------+---------+-----------------+
|APT_NO|  CREDIT_CARD_NO|   CUST_CITY| CUST_COUNTRY|         CUST_EMAIL|  CUST_PHONE|CUST_STATE|CUST_ZIP|FIRST_NAME|LAST_NAME|       LAST_UPDATED|MIDDLE_NAME|      SSN|      STREET_NAME|
+------+----------------+------------+-------------+-------------------+------------+----------+--------+----------+---------+-------------------+-----------+---------+-----------------+
|   656|4210653310061055|     Natchez|United States|AHooper@example.com|(555)1237818|        MS|   39120|      Alec|   Hooper|2018-04-21 12:49:02|         wm|123456100|Main Street North|
|   829|4210653310102868|Wethersfield|United States|EHolman@example.com|(555)1238933|        CT|    6109|      Etta|   Holman|2018-04-21 12:49:02|    brendan|123453023|    Redwood Drive|
+------+----------------+------------+-------------+-------------

In [123]:
customer_transformed_df.createOrReplaceTempView('customer_df')

In [125]:
open_query = session.sql("""
SELECT *
FROM 
customer_df
""")

In [126]:
open_query.show(2)

+------+----------------+------------+-------------+-------------------+------------+----------+--------+----------+---------+-------------------+-----------+---------+-----------------+
|APT_NO|  CREDIT_CARD_NO|   CUST_CITY| CUST_COUNTRY|         CUST_EMAIL|  CUST_PHONE|CUST_STATE|CUST_ZIP|FIRST_NAME|LAST_NAME|       LAST_UPDATED|MIDDLE_NAME|      SSN|      STREET_NAME|
+------+----------------+------------+-------------+-------------------+------------+----------+--------+----------+---------+-------------------+-----------+---------+-----------------+
|   656|4210653310061055|     Natchez|United States|AHooper@example.com|(555)1237818|        MS|   39120|      Alec|   Hooper|2018-04-21 12:49:02|         wm|123456100|Main Street North|
|   829|4210653310102868|Wethersfield|United States|EHolman@example.com|(555)1238933|        CT|    6109|      Etta|   Holman|2018-04-21 12:49:02|    brendan|123453023|    Redwood Drive|
+------+----------------+------------+-------------+-------------

In [131]:
concat = session.sql("""
SELECT *, CONCAT(STREET_NAME, ', ', APT_NO) AS FULL_STREET_ADDRESS
FROM customer_df
""")

In [132]:
concat.show(truncate=False)

+------+----------------+------------+-------------+----------------------+------------+----------+--------+----------+---------+-------------------+-----------+---------+-----------------+----------------------+
|APT_NO|CREDIT_CARD_NO  |CUST_CITY   |CUST_COUNTRY |CUST_EMAIL            |CUST_PHONE  |CUST_STATE|CUST_ZIP|FIRST_NAME|LAST_NAME|LAST_UPDATED       |MIDDLE_NAME|SSN      |STREET_NAME      |FULL_STREET_ADDRESS   |
+------+----------------+------------+-------------+----------------------+------------+----------+--------+----------+---------+-------------------+-----------+---------+-----------------+----------------------+
|656   |4210653310061055|Natchez     |United States|AHooper@example.com   |(555)1237818|MS        |39120   |Alec      |Hooper   |2018-04-21 12:49:02|wm         |123456100|Main Street North|Main Street North, 656|
|829   |4210653310102868|Wethersfield|United States|EHolman@example.com   |(555)1238933|CT        |6109    |Etta      |Holman   |2018-04-21 12:49:02

In [130]:
customer_transformed_df_backup = customer_transformed_df

In [140]:
customer_transformed_df = concat.drop('STREET_NAME')
customer_transformed_df = customer_transformed_df.drop('APT_NO')

In [142]:
customer_transformed_df.show(2, truncate=False)

+----------------+------------+-------------+-------------------+------------+----------+--------+----------+---------+-------------------+-----------+---------+----------------------+
|CREDIT_CARD_NO  |CUST_CITY   |CUST_COUNTRY |CUST_EMAIL         |CUST_PHONE  |CUST_STATE|CUST_ZIP|FIRST_NAME|LAST_NAME|LAST_UPDATED       |MIDDLE_NAME|SSN      |FULL_STREET_ADDRESS   |
+----------------+------------+-------------+-------------------+------------+----------+--------+----------+---------+-------------------+-----------+---------+----------------------+
|4210653310061055|Natchez     |United States|AHooper@example.com|(555)1237818|MS        |39120   |Alec      |Hooper   |2018-04-21 12:49:02|wm         |123456100|Main Street North, 656|
|4210653310102868|Wethersfield|United States|EHolman@example.com|(555)1238933|CT        |6109    |Etta      |Holman   |2018-04-21 12:49:02|brendan    |123453023|Redwood Drive, 829    |
+----------------+------------+-------------+-------------------+----------

In [186]:
# NOTE: pull in mapping doc csv's
# use pandas to grap the types and convert, clean this up later

def type_check(dataframe, data_types):
    schema_list = dataframe.schema.simpleString().split(",")
    #schema_list = customer_transformed_df.schema.simpleString().split(",")
    print(len(schema_list))
    print(len(data_types))
    for num, schema in enumerate(schema_list):

        
        type = schema.split(":")
        if type[1] != data_types[num]:
            print(f"Datatype mis-match: {type[0]} does not match required type, {data_types[num]}\n")
        else:
            print(f"{type[0]} matches required type {data_types[num]}\n")


In [ ]:
types = ['string', 'string', 'string', 'string', 'string', 'int', 'string', 'string', 'timestamp', 'string', 'int', 'string', 'string']
type_check(customer_transformed_df, types)

In [190]:
customer_transformed_df.printSchema()

root
 |-- CREDIT_CARD_NO: string (nullable = true)
 |-- CUST_CITY: string (nullable = true)
 |-- CUST_COUNTRY: string (nullable = true)
 |-- CUST_EMAIL: string (nullable = true)
 |-- CUST_PHONE: string (nullable = true)
 |-- CUST_STATE: string (nullable = true)
 |-- CUST_ZIP: long (nullable = true)
 |-- FIRST_NAME: string (nullable = true)
 |-- LAST_NAME: string (nullable = true)
 |-- LAST_UPDATED: timestamp (nullable = true)
 |-- MIDDLE_NAME: string (nullable = true)
 |-- SSN: long (nullable = true)
 |-- FULL_STREET_ADDRESS: string (nullable = true)



In [225]:
customer_transformed_df.coalesce(1).write.json(path='../data/credit_card_dataset_transformed/customer_transformed.json')
# NOTE:
# This took me WAY TOO LONG to figure out, pyspark documentation is def not as robust as pandas


In [269]:
requirements_df = pd.read_clipboard()  # read straight from requirements

In [270]:
requirements_df

,Source File Column Names,Target DataType
0,BRANCH_CODE,INT
1,BRANCH_NAME,VARCHAR
2,BRANCH_STREET,VARCHAR
3,BRANCH_CITY,VARCHAR
4,BRANCH_STATE,VARCHAR
5,BRANCH_ZIP,INT
6,BRANCH_PHONE,VARCHAR
7,LAST_UPDATED,TIMESTAMP


In [273]:
requirements_df.sort_values('Source File Column Names')

,Source File Column Names,Target DataType
3,BRANCH_CITY,VARCHAR
0,BRANCH_CODE,INT
1,BRANCH_NAME,VARCHAR
6,BRANCH_PHONE,VARCHAR
4,BRANCH_STATE,VARCHAR
2,BRANCH_STREET,VARCHAR
5,BRANCH_ZIP,INT
7,LAST_UPDATED,TIMESTAMP


In [5]:
branch_df = branch_df.withColumn('LAST_UPDATED', branch_df['LAST_UPDATED'].cast(TimestampType()))

In [6]:
branch_df.show(2)

+-----------+-----------+------------+------------+------------+-----------------+----------+-------------------+
|BRANCH_CITY|BRANCH_CODE| BRANCH_NAME|BRANCH_PHONE|BRANCH_STATE|    BRANCH_STREET|BRANCH_ZIP|       LAST_UPDATED|
+-----------+-----------+------------+------------+------------+-----------------+----------+-------------------+
|  Lakeville|          1|Example Bank|  1234565276|          MN|     Bridle Court|     55044|2018-04-18 16:51:47|
|    Huntley|          2|Example Bank|  1234618993|          IL|Washington Street|     60142|2018-04-18 16:51:47|
+-----------+-----------+------------+------------+------------+-----------------+----------+-------------------+
only showing top 2 rows



In [7]:
branch_df.printSchema()

root
 |-- BRANCH_CITY: string (nullable = true)
 |-- BRANCH_CODE: long (nullable = true)
 |-- BRANCH_NAME: string (nullable = true)
 |-- BRANCH_PHONE: string (nullable = true)
 |-- BRANCH_STATE: string (nullable = true)
 |-- BRANCH_STREET: string (nullable = true)
 |-- BRANCH_ZIP: long (nullable = true)
 |-- LAST_UPDATED: timestamp (nullable = true)



In [18]:
branch_df.createOrReplaceTempView('branch_df')

In [23]:
check = session.sql("""
SELECT *
FROM branch_df
WHERE BRANCH_ZIP is NULL

""")

In [24]:
branch_df = branch_df.fillna({"BRANCH_ZIP":99999})

In [29]:
branch_df_p = branch_df.toPandas()

c:\Users\GzuzC\OneDrive\Documents\Notes\Personal Studies\Workbooks and Worksheets\Python\PerScholas Data Engineering\perscholas_capstone_project\ignore\ETL-capstone-env\lib\site-packages\pyspark\sql\pandas\conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


In [31]:
branch_df_p.dtypes

BRANCH_CITY              object
BRANCH_CODE               int64
BRANCH_NAME              object
BRANCH_PHONE             object
BRANCH_STATE             object
BRANCH_STREET            object
BRANCH_ZIP                int64
LAST_UPDATED     datetime64[ns]
dtype: object

In [32]:
branch_df_p

,BRANCH_CITY,BRANCH_CODE,BRANCH_NAME,BRANCH_PHONE,BRANCH_STATE,BRANCH_STREET,BRANCH_ZIP,LAST_UPDATED
0,Lakeville,1,Example Bank,1234565276,MN,Bridle Court,55044,2018-04-18 16:51:47
1,Huntley,2,Example Bank,1234618993,IL,Washington Street,60142,2018-04-18 16:51:47
2,SouthRichmondHill,3,Example Bank,1234985926,NY,Warren Street,11419,2018-04-18 16:51:47
3,Middleburg,4,Example Bank,1234663064,FL,Cleveland Street,32068,2018-04-18 16:51:47
4,KingOfPrussia,5,Example Bank,1234849701,PA,14th Street,19406,2018-04-18 16:51:47
...,...,...,...,...,...,...,...,...
110,Baldwin,178,Example Bank,1234612707,NY,8th Street West,11510,2018-04-18 16:51:47
111,NewHaven,180,Example Bank,1234347181,CT,Meadow Street,6511,2018-04-18 16:51:47
112,Quincy,188,Example Bank,1234259065,MA,Lincoln Avenue,2169,2018-04-18 16:51:47
113,Lititz,192,Example Bank,1234937904,PA,Glenwood Drive,17543,2018-04-18 16:51:47


In [47]:
def parse_phone_number(phone_number):  # (xxx)xxx-xxxx
    phone_number = str(phone_number)
    if len(phone_number) == 10:
        phone_number = list(phone_number)
        phone_number.insert(0, "(")  # 0
        phone_number.insert(4, ")")  # 0, 1, 2 + 3 = 4
        phone_number.insert(-4, "-")
        return "".join(char for char in phone_number)
    else:
        pass


In [48]:
print(parse_phone_number("1234914637"))

(123)491-4637


In [50]:
branch_df_p['BRANCH_PHONE'] = branch_df_p['BRANCH_PHONE'].apply(parse_phone_number)

In [52]:
branch_df_p.head(10)

,BRANCH_CITY,BRANCH_CODE,BRANCH_NAME,BRANCH_PHONE,BRANCH_STATE,BRANCH_STREET,BRANCH_ZIP,LAST_UPDATED
0,Lakeville,1,Example Bank,(123)456-5276,MN,Bridle Court,55044,2018-04-18 16:51:47
1,Huntley,2,Example Bank,(123)461-8993,IL,Washington Street,60142,2018-04-18 16:51:47
2,SouthRichmondHill,3,Example Bank,(123)498-5926,NY,Warren Street,11419,2018-04-18 16:51:47
3,Middleburg,4,Example Bank,(123)466-3064,FL,Cleveland Street,32068,2018-04-18 16:51:47
4,KingOfPrussia,5,Example Bank,(123)484-9701,PA,14th Street,19406,2018-04-18 16:51:47
5,Paterson,7,Example Bank,(123)414-4890,NJ,Jefferson Street,7501,2018-04-18 16:51:47
6,Pittsford,8,Example Bank,(123)467-8272,NY,B Street,14534,2018-04-18 16:51:47
7,Wethersfield,9,Example Bank,(123)467-5219,CT,Jefferson Court,6109,2018-04-18 16:51:47
8,NorthOlmsted,10,Example Bank,(123)414-5047,OH,Cambridge Road,44070,2018-04-18 16:51:47
9,Hillsborough,11,Example Bank,(123)436-6354,NJ,3rd Street West,8844,2018-04-18 16:51:47


In [57]:
branch_df_p.to_json('../data/credit_card_dataset_transformed/transformed_branch.json', orient='records', lines=True)

In [58]:
branch_df = session.createDataFrame(branch_df_p)

c:\Users\GzuzC\OneDrive\Documents\Notes\Personal Studies\Workbooks and Worksheets\Python\PerScholas Data Engineering\perscholas_capstone_project\ignore\ETL-capstone-env\lib\site-packages\pyspark\sql\pandas\conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
c:\Users\GzuzC\OneDrive\Documents\Notes\Personal Studies\Workbooks and Worksheets\Python\PerScholas Data Engineering\perscholas_capstone_project\ignore\ETL-capstone-env\lib\site-packages\pyspark\sql\pandas\conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [59]:
branch_df.show(2)

+-----------+-----------+------------+-------------+------------+-----------------+----------+-------------------+
|BRANCH_CITY|BRANCH_CODE| BRANCH_NAME| BRANCH_PHONE|BRANCH_STATE|    BRANCH_STREET|BRANCH_ZIP|       LAST_UPDATED|
+-----------+-----------+------------+-------------+------------+-----------------+----------+-------------------+
|  Lakeville|          1|Example Bank|(123)456-5276|          MN|     Bridle Court|     55044|2018-04-18 16:51:47|
|    Huntley|          2|Example Bank|(123)461-8993|          IL|Washington Street|     60142|2018-04-18 16:51:47|
+-----------+-----------+------------+-------------+------------+-----------------+----------+-------------------+
only showing top 2 rows



In [62]:
credit_df.show(2)

+-----------+----------------+---------+---+-----+--------------+----------------+-----------------+----+
|BRANCH_CODE|  CREDIT_CARD_NO| CUST_SSN|DAY|MONTH|TRANSACTION_ID|TRANSACTION_TYPE|TRANSACTION_VALUE|YEAR|
+-----------+----------------+---------+---+-----+--------------+----------------+-----------------+----+
|        114|4210653349028689|123459988| 14|    2|             1|       Education|             78.9|2018|
|         35|4210653349028689|123459988| 20|    3|             2|   Entertainment|            14.24|2018|
+-----------+----------------+---------+---+-----+--------------+----------------+-----------------+----+
only showing top 2 rows



In [64]:
credit_df.printSchema()

root
 |-- BRANCH_CODE: long (nullable = true)
 |-- CREDIT_CARD_NO: string (nullable = true)
 |-- CUST_SSN: long (nullable = true)
 |-- DAY: long (nullable = true)
 |-- MONTH: long (nullable = true)
 |-- TRANSACTION_ID: long (nullable = true)
 |-- TRANSACTION_TYPE: string (nullable = true)
 |-- TRANSACTION_VALUE: double (nullable = true)
 |-- YEAR: long (nullable = true)



In [106]:
credit_df.createOrReplaceTempView("credit_df")  # YYYYMMDD

In [67]:
credit_df_p = credit_df.toPandas()

In [89]:
credit_df_p.dtypes

BRANCH_CODE            int64
CREDIT_CARD_NO        object
CUST_SSN               int64
DAY                    int64
MONTH                 object
TRANSACTION_ID         int64
TRANSACTION_TYPE      object
TRANSACTION_VALUE    float64
YEAR                   int64
dtype: object

In [93]:
credit_df_p['MONTH'] = credit_df_p['MONTH'].apply(lambda x: str( "0" + str(x)) if len(str(x)) < 2 else str(x))

In [94]:
credit_df_p.head(2)

,BRANCH_CODE,CREDIT_CARD_NO,CUST_SSN,DAY,MONTH,TRANSACTION_ID,TRANSACTION_TYPE,TRANSACTION_VALUE,YEAR
0,114,4210653349028689,123459988,14,02,1,Education,78.90,2018
1,35,4210653349028689,123459988,20,03,2,Entertainment,14.24,2018


In [100]:
credit_df = session.createDataFrame(credit_df_p)

c:\Users\GzuzC\OneDrive\Documents\Notes\Personal Studies\Workbooks and Worksheets\Python\PerScholas Data Engineering\perscholas_capstone_project\ignore\ETL-capstone-env\lib\site-packages\pyspark\sql\pandas\conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
c:\Users\GzuzC\OneDrive\Documents\Notes\Personal Studies\Workbooks and Worksheets\Python\PerScholas Data Engineering\perscholas_capstone_project\ignore\ETL-capstone-env\lib\site-packages\pyspark\sql\pandas\conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [105]:
credit_df.show(2)

+-----------+----------------+---------+---+-----+--------------+----------------+-----------------+----+
|BRANCH_CODE|  CREDIT_CARD_NO| CUST_SSN|DAY|MONTH|TRANSACTION_ID|TRANSACTION_TYPE|TRANSACTION_VALUE|YEAR|
+-----------+----------------+---------+---+-----+--------------+----------------+-----------------+----+
|        114|4210653349028689|123459988| 14|   02|             1|       Education|             78.9|2018|
|         35|4210653349028689|123459988| 20|   03|             2|   Entertainment|            14.24|2018|
+-----------+----------------+---------+---+-----+--------------+----------------+-----------------+----+
only showing top 2 rows



In [107]:
credit_df.createOrReplaceTempView("credit_df")

In [108]:
credit_df_concat = session.sql("""
    SELECT 
        *, 
        CONCAT(CAST(YEAR AS STRING), 
        CAST(MONTH AS STRING), 
        CAST(DAY AS STRING)) AS TIMEID
    FROM 
        credit_df
""")

In [109]:
credit_df_concat.show(2)

+-----------+----------------+---------+---+-----+--------------+----------------+-----------------+----+--------+
|BRANCH_CODE|  CREDIT_CARD_NO| CUST_SSN|DAY|MONTH|TRANSACTION_ID|TRANSACTION_TYPE|TRANSACTION_VALUE|YEAR|  TIMEID|
+-----------+----------------+---------+---+-----+--------------+----------------+-----------------+----+--------+
|        114|4210653349028689|123459988| 14|   02|             1|       Education|             78.9|2018|20180214|
|         35|4210653349028689|123459988| 20|   03|             2|   Entertainment|            14.24|2018|20180320|
+-----------+----------------+---------+---+-----+--------------+----------------+-----------------+----+--------+
only showing top 2 rows



In [180]:
credit_df_concat.printSchema()

root
 |-- BRANCH_CODE: long (nullable = true)
 |-- CREDIT_CARD_NO: string (nullable = true)
 |-- CUST_SSN: long (nullable = true)
 |-- DAY: long (nullable = true)
 |-- MONTH: string (nullable = true)
 |-- TRANSACTION_ID: long (nullable = true)
 |-- TRANSACTION_TYPE: string (nullable = true)
 |-- TRANSACTION_VALUE: double (nullable = true)
 |-- YEAR: long (nullable = true)
 |-- TIMEID: string (nullable = true)



In [197]:
for names in ['YEAR', 'MONTH', 'DAY']:
    credit_df_concat = credit_df_concat.drop(names)

In [198]:
credit_df_concat.printSchema()


root
 |-- BRANCH_CODE: long (nullable = true)
 |-- CREDIT_CARD_NO: string (nullable = true)
 |-- CUST_SSN: long (nullable = true)
 |-- TRANSACTION_ID: long (nullable = true)
 |-- TRANSACTION_TYPE: string (nullable = true)
 |-- TRANSACTION_VALUE: double (nullable = true)
 |-- TIMEID: string (nullable = true)



In [199]:
credit_df_concat_backup = credit_df_concat.toPandas()

In [ ]:
credit_df_concat_backup.to_json('../data/credit_card_dataset_transformed/credit_transformed.json', orient='records', lines=True)